In [2]:
import sys, os
sys.path.append(os.path.abspath('..'))
from graphilp.imports import ilpgraph, graph_formats
from graphilp.imports import networkx as imp_nx
import stp_flow_cut
import stpModel
import stp_flow_mtz
import stp_flow_mtz_L
import stp_flow_mtz_L_plus
import stp_flow_rolf
import networkx as nx
from gurobipy import *
import stpModel
import random
import xlsxwriter
import random
import matplotlib.pyplot as plt
import sys, os
%load_ext autoreload
%autoreload 2

In [9]:
filePath = os.path.join(os.getcwd(), 'testInstances', 'TooSmall', 'es10fst01.stp')
G, terminals = graph_formats.stp_to_networkx(filePath)
pos = dict()
pos[0] = (0,0)
for node in G.nodes(data=True):
    if 'coordinates' in node[1]:
        pos[node[0]] = node[1]['coordinates']
    else:                    
        pos[node[0]] = (random.randint(1,1000), random.randint(1,1000))
        
for edge in G.edges():
    G[edge[0]][edge[1]]['capacity'] = 1000

root = 1
Graph = imp_nx.read(G)

newModel = stp_flow_mtz.stpModelMtz(Graph, terminals, 'weight', 0, root, pos)
newModel.m.optimize()

variables = newModel.extractSolution(Graph)
vars = []
flows = []
#  Extract edge variables
print(newModel.G.node_variables)
for k, v in newModel.G.edge_variables.items():
    if v.X > 0.5:
        vars.append(v)

# Extract solution Edges
solutionEdges = [newModel.var2edge[var] for var in vars if (var in newModel.var2edge)]
print(solutionEdges)
# nodeSizes = []
# solutionGraph = nx.DiGraph()
# solutionGraph.add_edges_from(solutionEdges)
# plt.rcParams["figure.figsize"] = [15, 10]
# nx.draw_networkx_nodes(solutionGraph, pos, node_size = 200)
# nx.draw_networkx_labels(solutionGraph, pos, font_size = 12)
# nx.draw_networkx_edges(solutionGraph, pos, arrows=True, width = 1)
# plt.show()
# plt.savefig("graph.png", dpi=200)


Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 275 rows, 134 columns and 664 nonzeros
Model fingerprint: 0x591a8c17
Variable types: 0 continuous, 134 integer (58 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+04, 4e+06]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 2e+01]
Presolve removed 137 rows and 52 columns
Presolve time: 0.00s
Presolved: 138 rows, 82 columns, 373 nonzeros
Variable types: 0 continuous, 82 integer (43 binary)
Found heuristic solution: objective 2.696064e+07

Root relaxation: objective 2.263779e+07, 96 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2.2638e+07    0   15 2.6961e+07 2.2638e+07  16.0%     -    0s
H    0     0                    2.400005e+07 2.2638e+07  

In [ ]:
def __write_solution(solution:list):
    
    workbook = xlsxwriter.Workbook('BenchmarksLinear_Flow.xlsx')
    worksheet = workbook.add_worksheet()
    bold = workbook.add_format({'bold':True})
    solutionRows = len(solution)
    solutionColumns = len(solution[0])
    
    worksheet.write('A1', 'Instance', bold)
    worksheet.write('B1', 'Edges', bold)
    worksheet.write('C1', 'Nodes', bold)
    worksheet.write('D1', 'Terminals', bold)
    worksheet.write('E1', 'Runtime', bold)
    worksheet.write('F1', 'Objective Value', bold)
    worksheet.write('G1', 'Constraints', bold)
    worksheet.write('H1', 'MIP Gap', bold)
    worksheet.write('I1', 'Method', bold)

    for i in range(solutionRows):
        for j in range(solutionColumns):
            worksheet.write(i + 1, j, solution[i][j])
    
        
def runBenchmark():
    solutions = []
    i = 0
    basepath = './testInstances'
    paths = []
    for fname in os.listdir(basepath):
        path = os.path.join(os.getcwd(), basepath, fname)
        if os.path.isdir(path):
            # skip directories
            continue
        with open(path) as f:
            print(path)
            G, terminals = graph_formats.stp_to_networkx(path)
            pos = dict()
            pos[0] = (0,0)

            for node in G.nodes(data=True):
                if 'coordinates' in node[1]:
                    pos[node[0]] = node[1]['coordinates']
                else:                    
                    pos[node[0]] = (random.randint(1,1000), random.randint(1,1000))

            for edge in G.edges():
                G[edge[0]][edge[1]]['capacity'] = 1000

            root = terminals[0]
            Graph = imp_nx.read(G)
            
            print("Connected: ", nx.is_connected(G))
            print("Nodes: ", len(G.nodes()))
            print("Edges: ", len(G.edges()))
            print("Root: ", root)
            method = ""
            for i in range(1,2):
                if i == 0:
                    model, var2edge = stp_flow_mtz.stpModelMtz(Graph, terminals, 'weight', 0, root, pos)
                    method = "MTZ"
                elif i == 1:
                    model, var2edge = stp_flow_mtz_L.stpModelMtzL(Graph, terminals, 'weight', 0, root, pos)
                    method = "MTZ-L"
                elif i == 2:
                    model, var2edge = stp_flow_mtz_L_plus.stpModelMtzLPlus(Graph, terminals, 'weight', 0, root, pos)
                    method = "MTZ-L-Plus"
                elif i == 3:
                    model, var2edge = stp_flow_mtz.stpModelMtz(Graph, terminals, 'weight', 0, root, pos)
                    method = "Standard"
                model.params.TimeLimit = 600
                model.optimize()
                solutions.append((path, len(G.edges()), len(G.nodes()), len(terminals), model.runTime, model.objVal,\
                  model.numConstrs, model.MIPGap, method))

    __write_solution(solutions)         

if __name__ == '__main__':
    runBenchmark()

/home/tsauter/FlowStuff/./testInstances/hc10u.stp
Connected:  True
Nodes:  1024
Edges:  5120
Root:  1
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 35338 rows, 22528 columns and 138731 nonzeros
Model fingerprint: 0xbd569ec2
Variable types: 0 continuous, 22528 integer (11264 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Presolve removed 6687 rows and 532 columns
Presolve time: 0.13s
Presolved: 28651 rows, 21996 columns, 115110 nonzeros
Variable types: 0 continuous, 21996 integer (10742 binary)
Found heuristic solution: objective 736.0000000

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...



Root relaxation: interrupted, 7043 

KeyboardInterrupt: 

# MTZ Formulations

In [ ]:
filePath = os.path.join(os.getcwd(), 'testInstances', 'TooSmall','c16.stp')
G, terminals = graph_formats.stp_to_networkx(filePath)
pos = dict()
pos[0] = (0,0)

for node in G.nodes(data=True):
    if 'coordinates' in node[1]:
        pos[node[0]] = node[1]['coordinates']
    else:                    
        pos[node[0]] = (random.randint(1,1000), random.randint(1,1000))

for edge in G.edges():
    G[edge[0]][edge[1]]['capacity'] = 1000

root = terminals[0]
Graph = imp_nx.read(G)

model, var2edge = stp_flow_mtz_L.create_model(Graph, terminals, 'weight', 0, root, pos)
model.Params.Threads = 3

model.optimize() 
variables = model.getVars()
vars = []
#  Extract edge variables
for v in variables:
    if v.X > 0.5:
        print(v.varName)
        if v.VarName.__contains__("y"):
            vars.append(v)
print(vars)
# Extract solution Edges
solutionEdges = [var2edge[vars[i]] for i in range(len(vars)) if (vars[i].X > 0.5) and \
                (vars[i] in var2edge)]

print(solutionEdges)
solutionGraph = nx.DiGraph()
solutionGraph.add_edges_from(solutionEdges)
plt.rcParams["figure.figsize"] = [15,10]
nx.draw_networkx_nodes(solutionGraph, pos, node_size = 200)
nx.draw_networkx_labels(solutionGraph, pos, font_size = 12)
nx.draw_networkx_edges(solutionGraph, pos, arrows=True, width = 1)
plt.show()

Changed value of parameter Threads to 3
   Prev: 0  Min: 0  Max: 1024  Default: 0
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 3 threads
Optimize a model with 126500 rows, 51500 columns and 401889 nonzeros
Model fingerprint: 0xc61a8c65
Variable types: 0 continuous, 51500 integer (25500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 5e+02]
  RHS range        [1e+00, 5e+00]
Presolve removed 37613 rows and 556 columns
Presolve time: 0.91s
Presolved: 88887 rows, 50944 columns, 321026 nonzeros
Variable types: 0 continuous, 50944 integer (25444 binary)

Deterministic concurrent LP optimizer: primal and dual simplex (primal and dual model)
Showing first log only...

Concurrent spin time: 0.03s

Solved with dual simplex (primal model)

Root relaxation: objective 2.000000e+00, 333 iterations, 0.35 seconds
Total elapsed time = 8.47s

    Node

# Cut Formulation

In [ ]:
if (randomGraph):
    nodesAmount = 30
    edgesAmount = 50
    G = nx.gnm_random_graph(nodesAmount, edgesAmount)
    terminalsAmount  = 5
    terminals = random.sample(range(1, nodesAmount), terminalsAmount)
    root = random.sample(range(1, nodesAmount), 1)
    if root in terminals:
        raise ValueError("Root is in Terminals. Please try again...")
        
#root = 4 # bip42u
#root = 14 # es10fst01
#root = 14 # d03
#root =  # d01
#root = 3 # d04
#root = 841 # d07

NameError: name 'randomGraph' is not defined